In [1]:
import pandas as pd
import numpy as np
import torch
import sklearn

cuda_available = torch.cuda.is_available()

In [2]:
season2018 = pd.read_csv("Season2018.csv")
season2019 = pd.read_csv("Season2019.csv")
season2020 = pd.read_csv("Season2020.csv")

In [3]:
np.random.seed(42)
season2018_test = np.random.permutation(season2018["fixture_id"])[:48]
season2019_test = np.random.permutation(season2019["fixture_id"])[:48]
season2020_test = np.random.permutation(season2020["fixture_id"])[:48]

In [4]:
test_fixtures = np.append(np.append(season2018_test, season2019_test), season2020_test)

In [5]:
df = pd.read_csv("SaudiLeague.csv")

In [6]:
df.head()

,fixture_id,team,formation,coach,players,isHome,rating,opponent,opponent_formation,opponent_coach,...,opponent_last_against_performance,pts_difference,pts_after,opponent_pts_after,top_scorrer,main_gk,main_captain,opponent_top_scorrer,opponent_main_gk,opponent_main_captain
0,151695,Al-Ettifaq,4-2-3-1,L. Ramos,"R. M'Bolhi, 'R. Arias', 'Hussein Al Sayed', 'S...",1,7.00,Al-Raed,4-2-3-1,B. Hasi,...,6.72,0,1,1,1,1,0,1,0,0
1,151695,Al-Raed,4-2-3-1,B. Hasi,"'A. Doukha', 'H. Belkaroui', 'Mohammed Al Amri...",0,6.98,Al-Ettifaq,4-2-3-1,L. Ramos,...,6.90,0,1,1,1,0,0,1,1,0
2,151696,Al-Hazm,4-2-3-1,D. Isăilă,"'M. Asselah', 'Alemão', 'Khaled Al Barakah', '...",1,6.97,Al Wehda Club,4-4-2,Fábio Carille,...,7.02,0,1,1,0,0,0,1,0,0
3,151696,Al Wehda Club,4-4-2,Fábio Carille,"'Mohamed Awad', 'Osama Hawsawi', 'Renato Chave...",0,6.77,Al-Hazm,4-2-3-1,D. Isăilă,...,6.81,0,1,1,1,0,0,0,0,0
4,151697,Ohod,4-2-3-1,F. Arce,"'Z. Laaroubi', 'Hussein Abdul Ghani', 'Apodi',...",1,6.64,Al-Nassr,4-2-3-1,D. Carreño,...,7.07,0,0,3,1,0,0,0,0,0


In [7]:
df.columns

Index(['fixture_id', 'team', 'formation', 'coach', 'players', 'isHome',
       'rating', 'opponent', 'opponent_formation', 'opponent_coach',
       'opponent_players', 'opponent_rating', 'season', 'date', 'venue',
       'status', 'round', 'team_goals', 'opponent_goals', 'pts',
       'opponent_pts', 'result', 'last_performance',
       'opponent_last_performance', 'last_against_performance',
       'opponent_last_against_performance', 'pts_difference', 'pts_after',
       'opponent_pts_after', 'top_scorrer', 'main_gk', 'main_captain',
       'opponent_top_scorrer', 'opponent_main_gk', 'opponent_main_captain'],
      dtype='object')

Let's perform all preprocessing steps then export the data into seperate CSVs

* What actions do we want to make on the data before modeling?
    1. Drop unnecessary columns 
    2. Scale numeric variables 
    3. Create dummy variables for categorical variables

##### 1.  dropping columns (columns that are not significant or can not be collected before the match)

In [8]:
df.columns

Index(['fixture_id', 'team', 'formation', 'coach', 'players', 'isHome',
       'rating', 'opponent', 'opponent_formation', 'opponent_coach',
       'opponent_players', 'opponent_rating', 'season', 'date', 'venue',
       'status', 'round', 'team_goals', 'opponent_goals', 'pts',
       'opponent_pts', 'result', 'last_performance',
       'opponent_last_performance', 'last_against_performance',
       'opponent_last_against_performance', 'pts_difference', 'pts_after',
       'opponent_pts_after', 'top_scorrer', 'main_gk', 'main_captain',
       'opponent_top_scorrer', 'opponent_main_gk', 'opponent_main_captain'],
      dtype='object')

In [9]:
df.drop(columns=["players", "rating", "opponent_players", "opponent_rating", "date", "venue",
                'status', "result", "pts_after","opponent_pts_after"], inplace=True)

In [10]:
df.columns

Index(['fixture_id', 'team', 'formation', 'coach', 'isHome', 'opponent',
       'opponent_formation', 'opponent_coach', 'season', 'round', 'team_goals',
       'opponent_goals', 'pts', 'opponent_pts', 'last_performance',
       'opponent_last_performance', 'last_against_performance',
       'opponent_last_against_performance', 'pts_difference', 'top_scorrer',
       'main_gk', 'main_captain', 'opponent_top_scorrer', 'opponent_main_gk',
       'opponent_main_captain'],
      dtype='object')

##### 2. Numeric variables scaling (based on metrics from the training data)

In [11]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

#####  let's define the training data to use it for fitting the scaler 

In [57]:
train_data = df.loc[~df["fixture_id"].isin(test_fixtures)]

In [58]:
train_data.shape

(1152, 25)

In [60]:
train_data[["round", "pts", "opponent_pts", "last_performance",
            "opponent_last_performance",'last_against_performance', 'opponent_last_against_performance',
            'pts_difference',]] = min_max_scaler.fit_transform(train_data[["round", "pts", "opponent_pts",
                                                      "last_performance","opponent_last_performance",
                                                      'last_against_performance', 'pts_difference',
                                                      'opponent_last_against_performance']])

/Users/yazeed/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/yazeed/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [51]:
train_data.head()

,fixture_id,team,formation,coach,isHome,opponent,opponent_formation,opponent_coach,season,round,...,opponent_last_performance,last_against_performance,opponent_last_against_performance,pts_difference,top_scorrer,main_gk,main_captain,opponent_top_scorrer,opponent_main_gk,opponent_main_captain
0,151695,Al-Ettifaq,4-2-3-1,L. Ramos,1,Al-Raed,4-2-3-1,B. Hasi,2018/2019,0.0,...,0.558140,0.610465,0.5,0.505814,1,1,0,1,0,0
1,151695,Al-Raed,4-2-3-1,B. Hasi,0,Al-Ettifaq,4-2-3-1,L. Ramos,2018/2019,0.0,...,0.517442,0.505814,0.5,0.610465,1,0,0,1,1,0
2,151696,Al-Hazm,4-2-3-1,D. Isăilă,1,Al Wehda Club,4-4-2,Fábio Carille,2018/2019,0.0,...,0.587209,0.558140,0.5,0.680233,0,0,0,1,0,0
3,151696,Al Wehda Club,4-4-2,Fábio Carille,0,Al-Hazm,4-2-3-1,D. Isăilă,2018/2019,0.0,...,0.563953,0.680233,0.5,0.558140,1,0,0,0,0,0
4,151697,Ohod,4-2-3-1,F. Arce,1,Al-Nassr,4-2-3-1,D. Carreño,2018/2019,0.0,...,0.674419,0.389535,0.5,0.709302,1,0,0,0,0,0


In [61]:
df[["round", "pts", "opponent_pts", "last_performance",
            "opponent_last_performance",'last_against_performance', 'opponent_last_against_performance',
            'pts_difference']] = min_max_scaler.transform(df[["round", "pts", "opponent_pts",
                                                      "last_performance","opponent_last_performance",
                                                      'last_against_performance', 'pts_difference',
                                                      'opponent_last_against_performance']])

In [62]:
df.head()

,fixture_id,team,formation,coach,isHome,opponent,opponent_formation,opponent_coach,season,round,...,opponent_last_performance,last_against_performance,opponent_last_against_performance,pts_difference,top_scorrer,main_gk,main_captain,opponent_top_scorrer,opponent_main_gk,opponent_main_captain
0,151695,Al-Ettifaq,4-2-3-1,L. Ramos,1,Al-Raed,4-2-3-1,B. Hasi,2018/2019,0.0,...,0.558140,0.610465,0.5,0.505814,1,1,0,1,0,0
1,151695,Al-Raed,4-2-3-1,B. Hasi,0,Al-Ettifaq,4-2-3-1,L. Ramos,2018/2019,0.0,...,0.517442,0.505814,0.5,0.610465,1,0,0,1,1,0
2,151696,Al-Hazm,4-2-3-1,D. Isăilă,1,Al Wehda Club,4-4-2,Fábio Carille,2018/2019,0.0,...,0.587209,0.558140,0.5,0.680233,0,0,0,1,0,0
3,151696,Al Wehda Club,4-4-2,Fábio Carille,0,Al-Hazm,4-2-3-1,D. Isăilă,2018/2019,0.0,...,0.563953,0.680233,0.5,0.558140,1,0,0,0,0,0
4,151697,Ohod,4-2-3-1,F. Arce,1,Al-Nassr,4-2-3-1,D. Carreño,2018/2019,0.0,...,0.674419,0.389535,0.5,0.709302,1,0,0,0,0,0


##### 3. dummy variables

In [63]:
df.columns

Index(['fixture_id', 'team', 'formation', 'coach', 'isHome', 'opponent',
       'opponent_formation', 'opponent_coach', 'season', 'round', 'team_goals',
       'opponent_goals', 'pts', 'opponent_pts', 'last_performance',
       'opponent_last_performance', 'last_against_performance',
       'opponent_last_against_performance', 'pts_difference', 'top_scorrer',
       'main_gk', 'main_captain', 'opponent_top_scorrer', 'opponent_main_gk',
       'opponent_main_captain'],
      dtype='object')

In [66]:
df.shape

(1440, 25)

In [64]:
dummy = pd.get_dummies(df[['team', 'formation', "coach", "opponent",
                           "opponent_formation", "opponent_coach","season"]])

In [65]:
dummy

,team_Abha,team_Al Baten,team_Al Shabab,team_Al Taawon,team_Al Wehda Club,team_Al-Adalah,team_Al-Ahli Jeddah,team_Al-Ain,team_Al-Ettifaq,team_Al-Faisaly FC,...,opponent_coach_V. Milojević,opponent_coach_Vítor Campelos,opponent_coach_Y. Al Mannai,opponent_coach_Y. Ferrera,opponent_coach_Yousef Alghadeer,opponent_coach_Youssef Anbar,opponent_coach_Z. Mamić,season_2018/2019,season_2019/2020,season_2020/2021
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1436,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1437,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [68]:
df = pd.concat([df, dummy], axis=1)

In [70]:
df.drop(columns=['team', 'formation', "coach", "opponent",
                           "opponent_formation", "opponent_coach","season"], inplace=True)

In [71]:
df.shape

(1440, 259)

In [73]:
train_data = df.loc[~df["fixture_id"].isin(test_fixtures)]
test_data =  df.loc[df["fixture_id"].isin(test_fixtures)]

In [75]:
train_data.to_csv("train.csv", index=False)
test_data.to_csv("test.csv", index=False)

In [18]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

We have to write custom Dataset class so that pytorch's Dataloader can accpet it

In [77]:
df.columns[:20]

Index(['fixture_id', 'isHome', 'round', 'team_goals', 'opponent_goals', 'pts',
       'opponent_pts', 'last_performance', 'opponent_last_performance',
       'last_against_performance', 'opponent_last_against_performance',
       'pts_difference', 'top_scorrer', 'main_gk', 'main_captain',
       'opponent_top_scorrer', 'opponent_main_gk', 'opponent_main_captain',
       'team_Abha', 'team_Al Baten'],
      dtype='object')

,fixture_id,isHome,round,team_goals,opponent_goals,pts,opponent_pts,last_performance,opponent_last_performance,last_against_performance,...,opponent_coach_V. Milojević,opponent_coach_Vítor Campelos,opponent_coach_Y. Al Mannai,opponent_coach_Y. Ferrera,opponent_coach_Yousef Alghadeer,opponent_coach_Youssef Anbar,opponent_coach_Z. Mamić,season_2018/2019,season_2019/2020,season_2020/2021
0,151695,1,0.0,1.0,1.0,0.000000,0.000000,0.517442,0.558140,0.610465,...,0,0,0,0,0,0,0,1,0,0
1,151695,0,0.0,1.0,1.0,0.000000,0.000000,0.558140,0.517442,0.505814,...,0,0,0,0,0,0,0,1,0,0
2,151696,1,0.0,0.0,0.0,0.000000,0.000000,0.563953,0.587209,0.558140,...,0,0,0,0,0,0,0,1,0,0
3,151696,0,0.0,0.0,0.0,0.000000,0.000000,0.587209,0.563953,0.680233,...,0,0,0,0,0,0,0,1,0,0
4,151697,1,0.0,1.0,2.0,0.000000,0.000000,0.505814,0.674419,0.389535,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,657879,0,1.0,0.0,3.0,0.463768,0.782609,0.383721,0.825581,0.406977,...,0,0,0,0,0,0,0,0,0,1
1148,657881,1,1.0,3.0,2.0,0.840580,0.521739,0.662791,0.343023,0.616279,...,0,0,0,0,0,0,0,0,0,1
1149,657881,0,1.0,2.0,3.0,0.521739,0.840580,0.343023,0.662791,0.645349,...,0,0,0,0,0,0,0,0,0,1
1150,657882,1,1.0,1.0,2.0,0.666667,0.768116,0.651163,0.668605,0.529070,...,0,0,0,0,0,0,0,0,0,1


In [200]:
from torch.utils.data import  Dataset
class SaudiLeagueDataset(torch.utils.data.Dataset):
    
    def __init__(self, file_path):
        
        self.data = pd.read_csv(file_path)
        self.data.drop("fixture_id", axis=1, inplace=True)
        self.data.drop(np.where(self.data['team_goals']>3)[0], inplace=True)
        self.data.reset_index(inplace=True, drop=True)
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)
        # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
        # in this example, i don't use ToTensor() method of torchvision.transforms
        # so you can convert numpy ndarray shape to tensor in PyTorch (H, W, C) --> (C, H, W)
        
        outcome = self.data.loc[index, ["team_goals"]].values.astype(np.float32)

        match = self.data.drop(columns=["team_goals", "opponent_goals"]).iloc[index].values.astype(np.float32)

            
        return match, outcome

In [201]:
train_dataset = SaudiLeagueDataset("train.csv")
test_dataset = SaudiLeagueDataset("test.csv")

In [202]:
train_dataset[0]

(array([1.        , 0.        , 0.        , 0.        , 0.51744187,
        0.55813956, 0.6104651 , 0.5       , 0.50581396, 1.        ,
        1.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

### perfect!

In [203]:
len(train_dataset)

1064

In [204]:
len(test_dataset)

263

In [205]:
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

train_size = 1064
indices = list(range(train_size))
np.random.shuffle(indices)

split = int(np.floor(train_size * 0.15)) #get 15% of the training data
train_idx, valid_idx = indices[split:], indices[:split]

batch_size = 32

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

loaders = {'train':DataLoader(train_dataset, batch_size=batch_size,
    sampler=train_sampler, num_workers=0),
                   'valid': DataLoader(train_dataset, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=0),
                   'test': DataLoader(test_dataset, batch_size=batch_size, 
                                          num_workers=0, shuffle=True, pin_memory=True)}

In [206]:
for match, outcome in loaders["valid"]:
    print(match)
    print(outcome)
    break

tensor([[1.0000, 0.0345, 0.0000,  ..., 1.0000, 0.0000, 0.0000],
        [1.0000, 0.1724, 0.0870,  ..., 0.0000, 0.0000, 1.0000],
        [0.0000, 1.0000, 0.7681,  ..., 0.0000, 0.0000, 1.0000],
        ...,
        [1.0000, 0.6207, 0.4783,  ..., 0.0000, 1.0000, 0.0000],
        [0.0000, 0.4138, 0.2029,  ..., 1.0000, 0.0000, 0.0000],
        [0.0000, 0.1724, 0.1304,  ..., 0.0000, 0.0000, 1.0000]])
tensor([[1.],
        [1.],
        [2.],
        [0.],
        [0.],
        [2.],
        [1.],
        [0.],
        [2.],
        [1.],
        [1.],
        [1.],
        [2.],
        [1.],
        [3.],
        [2.],
        [3.],
        [0.],
        [1.],
        [0.],
        [2.],
        [1.],
        [2.],
        [1.],
        [3.],
        [0.],
        [1.],
        [1.],
        [2.],
        [0.],
        [3.],
        [1.]])


In [282]:
train_dataset[0][0].shape

(256,)

In [289]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 16)
        self.fc5 = nn.Linear(16, 8)
#         self.fc6 = nn.Linear(8, 4)
        self.fc6 = nn.Linear(8, 1)
        
        self.dropout = nn.Dropout(0.4)
        
    def forward(self, x):
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = F.relu(self.fc4(x))
        x = self.dropout(x)
        x = F.relu(self.fc5(x))
#         x = self.dropout(x)
#         x = F.relu(self.fc6(x))
        x = self.dropout(x)
    
        x = self.fc6(x)

        return x

model = Net()
print(model)

if cuda_available:
    model.cuda()

Net(
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (fc5): Linear(in_features=16, out_features=8, bias=True)
  (fc6): Linear(in_features=8, out_features=1, bias=True)
  (dropout): Dropout(p=0.4, inplace=False)
)


In [345]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(256, 170)
        self.fc2 = nn.Linear(170, 113)
        self.fc3 = nn.Linear(113, 75)
        self.fc4 = nn.Linear(75, 50)
        self.fc5 = nn.Linear(50, 33)
        self.fc6 = nn.Linear(33, 22)
        self.fc7 = nn.Linear(22, 14)
        self.fc8 = nn.Linear(14, 9)
        self.fc9 = nn.Linear(9, 6)
        self.fc10 = nn.Linear(6, 1)
        
        self.dropout = nn.Dropout(0.6)
        
    def forward(self, x):
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = F.relu(self.fc4(x))
        x = self.dropout(x)
        x = F.relu(self.fc5(x))
        x = self.dropout(x)
        x = F.relu(self.fc6(x))
        x = self.dropout(x)
        x = F.relu(self.fc7(x))
        x = self.dropout(x)
        x = F.relu(self.fc8(x))
        x = self.dropout(x)
        x = F.relu(self.fc9(x))
        x = self.dropout(x)

        x = self.fc10(x)

        return x

model = Net()
print(model)

if cuda_available:
    model.cuda()

Net(
  (fc1): Linear(in_features=256, out_features=170, bias=True)
  (fc2): Linear(in_features=170, out_features=113, bias=True)
  (fc3): Linear(in_features=113, out_features=75, bias=True)
  (fc4): Linear(in_features=75, out_features=50, bias=True)
  (fc5): Linear(in_features=50, out_features=33, bias=True)
  (fc6): Linear(in_features=33, out_features=22, bias=True)
  (fc7): Linear(in_features=22, out_features=14, bias=True)
  (fc8): Linear(in_features=14, out_features=9, bias=True)
  (fc9): Linear(in_features=9, out_features=6, bias=True)
  (fc10): Linear(in_features=6, out_features=1, bias=True)
  (dropout): Dropout(p=0.6, inplace=False)
)


In [346]:
criterion = torch.nn.MSELoss()


optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)


As we train we'll keep an eye on the validation loss

In [347]:
# number of epochs to train the model
n_epochs = 100

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for matches, outcomes in loaders["train"]:
        # move tensors to GPU if CUDA is available
        if cuda_available:
            matches, outcomes = matches.cuda(), outcomes.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(matches)
        # calculate the batch loss
        loss = criterion(output, outcomes)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*matches.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for matches, outcomes in loaders["valid"]:
        # move tensors to GPU if CUDA is available
        if cuda_available: 
            matches, outcomes = matches.cuda(), outcomes.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(matches)
        # calculate the batch loss
        loss = criterion(output, outcomes)
        # update average validation loss 
        valid_loss += loss.item()*matches.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(loaders["train"].dataset)
    valid_loss = valid_loss/len(loaders["valid"].dataset)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 1.755447 	Validation Loss: 0.298532
Validation loss decreased (inf --> 0.298532).  Saving model ...
Epoch: 2 	Training Loss: 1.628073 	Validation Loss: 0.276531
Validation loss decreased (0.298532 --> 0.276531).  Saving model ...
Epoch: 3 	Training Loss: 1.513330 	Validation Loss: 0.257593
Validation loss decreased (0.276531 --> 0.257593).  Saving model ...
Epoch: 4 	Training Loss: 1.408167 	Validation Loss: 0.242211
Validation loss decreased (0.257593 --> 0.242211).  Saving model ...
Epoch: 5 	Training Loss: 1.331793 	Validation Loss: 0.229255
Validation loss decreased (0.242211 --> 0.229255).  Saving model ...
Epoch: 6 	Training Loss: 1.264035 	Validation Loss: 0.218304
Validation loss decreased (0.229255 --> 0.218304).  Saving model ...
Epoch: 7 	Training Loss: 1.201668 	Validation Loss: 0.208443
Validation loss decreased (0.218304 --> 0.208443).  Saving model ...
Epoch: 8 	Training Loss: 1.145523 	Validation Loss: 0.199858
Validation loss decreased (0.20844

Epoch: 64 	Training Loss: 0.755857 	Validation Loss: 0.134978
Validation loss decreased (0.134992 --> 0.134978).  Saving model ...
Epoch: 65 	Training Loss: 0.756657 	Validation Loss: 0.134967
Validation loss decreased (0.134978 --> 0.134967).  Saving model ...
Epoch: 66 	Training Loss: 0.758283 	Validation Loss: 0.134951
Validation loss decreased (0.134967 --> 0.134951).  Saving model ...
Epoch: 67 	Training Loss: 0.757086 	Validation Loss: 0.134947
Validation loss decreased (0.134951 --> 0.134947).  Saving model ...
Epoch: 68 	Training Loss: 0.758786 	Validation Loss: 0.134935
Validation loss decreased (0.134947 --> 0.134935).  Saving model ...
Epoch: 69 	Training Loss: 0.753675 	Validation Loss: 0.134933
Validation loss decreased (0.134935 --> 0.134933).  Saving model ...


KeyboardInterrupt: 

In [348]:
print(valid_loss_min)

0.13493284230169497


In [349]:
model.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [350]:
test_loss = 0.0
correct = 0
count=0

model.eval()
# iterate over test data
for matches, outcomes in loaders["test"]:
    # move tensors to GPU if CUDA is available
    if cuda_available:
        matches, outcomes  = matches.cuda(), outcomes.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = torch.round(model(matches))

    # calculate the batch loss
    loss = criterion(output, outcomes)
    # update test loss 
    test_loss += loss.item()*matches.size(0)
    try:
        for i in range(batch_size):
            result = outcomes.data[i]
            if torch.round(output[i]).eq(result):
                correct+=1
            count+=1
    except:
        pass
    
# average test loss
test_loss = test_loss/len(loaders["test"].dataset)
print('Test Loss: {}\n'.format(test_loss))
print('Test accuracy: {}\n'.format((correct / len(loaders["test"].dataset))*100))


Test Loss: 1.0114068454662657

Test accuracy: 36.50190114068441



## Trying out different regression models

train_data["result"] = ""
test_data["result"] = ""

for i in train_data.index:
    if train_data.loc[i, "team_goals"] > train_data.loc[i, "opponent_goals"]:
        
        train_data.loc[i, "result"] = "0"
    elif train_data.loc[i, "team_goals"] < train_data.loc[i, "opponent_goals"]:
        train_data.loc[i, "result"] = '1'
    else:
        train_data.loc[i, "result"] = '2'

for i in test_data.index:
    if test_data.loc[i, "team_goals"] > test_data.loc[i, "opponent_goals"]:
        
        test_data.loc[i, "result"] = '0'
    elif test_data.loc[i, "team_goals"] < test_data.loc[i, "opponent_goals"]:
        test_data.loc[i, "result"] = '1'
    else:
        test_data.loc[i, "result"] = '2'


train_data.drop()
test



In [358]:
from sklearn.metrics import mean_squared_error, accuracy_score

##### 1. Random forest regressor

In [373]:
train_data.drop(np.where(train_data["team_goals"]>3)[0], inplace=True)
test_data.drop(np.where(test_data["team_goals"]>3)[0], inplace=True)

In [372]:
from sklearn.ensemble import RandomForestRegressor

In [374]:
model_rf = RandomForestRegressor()

In [375]:
model_rf.fit(train_data.drop(columns=["fixture_id", "team_goals", 'opponent_goals']), train_data["team_goals"])

RandomForestRegressor()

In [376]:
preds_rf = np.round(model_rf.predict(test_data.drop(columns=["fixture_id", "team_goals", 'opponent_goals'])))

In [377]:
mean_squared_error(test_data.team_goals, preds_rf)

1.0114068441064639

In [378]:
accuracy_score(test_data.team_goals, preds_rf)

0.35361216730038025

In [379]:
model_sv = sklearn.svm.SVR()

In [380]:
model_sv.fit(train_data.drop(columns=["fixture_id", "team_goals", 'opponent_goals']), train_data["team_goals"])

SVR()

In [381]:
preds_sv = np.round(model_sv.predict(test_data.drop(columns=["fixture_id", "team_goals", 'opponent_goals'])))

In [382]:
mean_squared_error(test_data.team_goals, preds_sv)

1.0190114068441065

In [383]:
accuracy_score(test_data.team_goals, preds_sv)

0.33460076045627374

In [385]:
from sklearn.linear_model import LogisticRegression

In [386]:
model_lr = LogisticRegression()

In [387]:
model_lr.fit(train_data.drop(columns=["fixture_id", "team_goals", 'opponent_goals']), train_data["team_goals"])

/Users/yazeed/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [389]:
preds_lr = np.round(model_lr.predict(test_data.drop(columns=["fixture_id", "team_goals", 'opponent_goals'])))

In [390]:
mean_squared_error(test_data.team_goals, preds_lr)

1.3231939163498099

In [391]:
accuracy_score(test_data.team_goals, preds_lr)

0.35361216730038025

<br>

In [392]:
from sklearn.kernel_ridge import KernelRidge

In [589]:
model_krr = KernelRidge(alpha=0.2, kernel='polynomial', degree=1, coef0=1)


In [590]:
model_krr.fit(train_data.drop(columns=["fixture_id", "team_goals", 'opponent_goals']), train_data["team_goals"])

KernelRidge(alpha=0.2, degree=1, kernel='polynomial')

In [591]:
preds_krr = np.round(model_krr.predict(test_data.drop(columns=["fixture_id", "team_goals", 'opponent_goals'])))

In [592]:
mean_squared_error(test_data.team_goals, preds_krr)

1.015209125475285

In [593]:
accuracy_score(test_data.team_goals, preds_krr)

0.3840304182509506

In [597]:
from sklearn.linear_model import ElasticNet

In [686]:
model_en = ElasticNet(alpha=0.005, l1_ratio=.86)

In [683]:
model_en.fit(train_data.drop(columns=["fixture_id", "team_goals", 'opponent_goals']), train_data["team_goals"])

ElasticNet(alpha=0.005, l1_ratio=0.86)

In [684]:
preds_en = np.round(model_en.predict(test_data.drop(columns=["fixture_id", "team_goals", 'opponent_goals'])))

In [685]:
accuracy_score(test_data.team_goals, preds_en)

0.3840304182509506

In [689]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
{‘friedman_mse’, ‘mse’, ‘mae’}

In [888]:
model_gb = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=16, min_samples_split=30, 
                                   loss='huber', random_state =5)

In [889]:
model_gb.fit(train_data.drop(columns=["fixture_id", "team_goals", 'opponent_goals']), train_data["team_goals"])

GradientBoostingRegressor(learning_rate=0.01, loss='huber', max_depth=4,
                          max_features='sqrt', min_samples_leaf=16,
                          min_samples_split=30, n_estimators=1000,
                          random_state=5)

In [890]:
preds_gb = np.round(model_gb.predict(test_data.drop(columns=["fixture_id", "team_goals", 'opponent_goals'])))

In [891]:
accuracy_score(test_data.team_goals, preds_gb)

0.39923954372623577

In [893]:
import xgboost as xgb

In [1242]:
model_xgb = xgb.XGBRegressor(base_score = 0.5, colsample_bytree=0.466, gamma=0.01, 
                             learning_rate=0.01, max_depth=3, 
                             min_child_weight=5, n_estimators=980,
                             reg_alpha=0.5, reg_lambda=0.05,
                             subsample=0.72, random_state =5, nthread = 2)

In [1243]:
model_xgb.fit(train_data.drop(columns=["fixture_id", "team_goals", 'opponent_goals']), train_data["team_goals"])

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.466, gamma=0.01, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=3,
             min_child_weight=5, missing=nan, monotone_constraints='()',
             n_estimators=980, n_jobs=2, nthread=2, num_parallel_tree=1,
             random_state=5, reg_alpha=0.5, reg_lambda=0.05, scale_pos_weight=1,
             subsample=0.72, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [1244]:
preds_xgb = np.round(model_xgb.predict(test_data.drop(columns=["fixture_id", "team_goals", 'opponent_goals'])))

In [1245]:
accuracy_score(test_data.team_goals, preds_xgb)

0.41064638783269963